# Distributed Processing Basics
Due to a number of limitations involving data passed to processes using `multiprocessing.Pool()`, I've implemented similar (although probably less robust) class called `Distribute()`.

In [1]:
import sys
sys.path.append('..')
import doctable

## `.map_chunk()` Method
Allows you to write map functions that processes a chunk of your data at a time. This is the lowest-level method for distributed processing.

In [2]:
# and now additional args plus the database
def muli_multi(nums):
    return [num*1.275 for num in nums]

nums = list(range(1000))
with doctable.Distribute(3) as d:
    %time res = d.map_chunk(muli_multi, nums)

with doctable.Distribute(1) as d: # won't create new process at all
    %time res = d.map_chunk(muli_multi, nums)
res[:3]

CPU times: user 6.72 ms, sys: 10.5 ms, total: 17.2 ms
Wall time: 19.6 ms
CPU times: user 240 µs, sys: 0 ns, total: 240 µs
Wall time: 249 µs


[0.0, 1.275, 2.55]

## `map_insert()` Method
Allows you to write methods which are meant to store single rows into a database. Note how `muli_multi_store()` inserts into database a single element, and the doctable is passed using the `dt_inst` keyword parameter.

In [3]:
db = doctable.DocTable(schema=(
    ('idcol', 'id'), 
    ('float', 'num', dict(unique=True)), 
), fname='tmp_distributed_basics.db')

# inserts result into db instead of returning
def muli_multi_store(num, db):
    db.insert({'num': num*1.275}, ifnotunique='replace')

with doctable.Distribute(2) as d:
    %time res = d.map_insert(muli_multi_store, nums, dt_inst=db)
db.select_df(limit=10)

CPU times: user 9.66 ms, sys: 11.7 ms, total: 21.3 ms
Wall time: 1.79 s


,id,num
0,2001,0.000
1,2002,1.275
2,2003,2.550
3,2004,3.825
4,2005,5.100
5,2006,6.375
6,2007,7.650
7,2008,8.925
8,2009,637.500
9,2010,638.775
